## Step 1: Load Existing Model

In [1]:
from keras.models import load_model, Sequential
from keras.layers import Dropout, LSTM, Dense
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
model = load_model('ASR_model.keras')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 100, 128)          39936     
 al)                                                             
                                                                 
 time_distributed (TimeDist  (None, 100, 100)          12900     
 ributed)                                                        
                                                                 
Total params: 52836 (206.39 KB)
Trainable params: 52836 (206.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Step 2: Adjust Architecture

In [3]:
model_new = Sequential()

for layer in model.layers[:-1]:
    model_new.add(layer)
    if isinstance(layer, LSTM):
        # Add dropout after each LSTM layer
        model_new.add(Dropout(0.5))

model_new.add(Dense(100, activation='softmax'))

## Step 3: Prepare Data

In [4]:
X = np.load('audio.npy')
y = np.load('labels.npy')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=44)

## Step 4: Compile and Retrain Model

In [5]:
model_new.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
history = model_new.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

Epoch 1/10
3/3 [==============================] - 7s 717ms/step - loss: 1.9726 - accuracy: 0.0000e+00 - val_loss: 1.8434 - val_accuracy: 0.0000e+00
Epoch 2/10
3/3 [==============================] - 0s 131ms/step - loss: 1.8127 - accuracy: 0.0000e+00 - val_loss: 1.7056 - val_accuracy: 0.0000e+00
Epoch 3/10
3/3 [==============================] - 0s 119ms/step - loss: 1.6794 - accuracy: 0.0019 - val_loss: 1.5901 - val_accuracy: 0.0255
Epoch 4/10
3/3 [==============================] - 0s 119ms/step - loss: 1.5680 - accuracy: 0.0275 - val_loss: 1.4954 - val_accuracy: 0.0460
Epoch 5/10
3/3 [==============================] - 0s 127ms/step - loss: 1.4778 - accuracy: 0.0435 - val_loss: 1.4217 - val_accuracy: 0.0400
Epoch 6/10
3/3 [==============================] - 1s 196ms/step - loss: 1.4102 - accuracy: 0.0430 - val_loss: 1.3700 - val_accuracy: 0.0400
Epoch 7/10
3/3 [==============================] - 1s 189ms/step - loss: 1.3634 - accuracy: 0.0434 - val_loss: 1.3398 - val_accuracy: 0.0405
Epoc

## Step 5: Evaluate Adjusted Model

In [7]:
performance = model_new.evaluate(X_val, y_val)

1/1 [==============================] - 0s 88ms/step - loss: 1.2992 - accuracy: 0.0380


In [8]:
print(f'Validation Loss: {performance[0]}, Validation Accuracy: {performance[1]}')

Validation Loss: 1.2991859912872314, Validation Accuracy: 0.03799999877810478


## Step 6: Save New Model

In [9]:
model_new.save('New_ASR_model.keras')